## Produção da tabela Recipes

In [2]:
import pandas as pd

df_recipes = pd.read_csv('../data/raw/culinarydb/01_Recipe_Details.csv', sep=',', header=0, encoding='UTF-8')

df_recipes

,Recipe ID,Title,Source,Cuisine
0,1,5 spice vegetable fried rice,TARLA_DALAL,Indian Subcontinent
1,2,aachar aaloo,TARLA_DALAL,Indian Subcontinent
2,3,aadu lassan keri nu athanu,TARLA_DALAL,Indian Subcontinent
3,4,aaloo kofta,TARLA_DALAL,Indian Subcontinent
4,5,aaloo tamatar subzi,TARLA_DALAL,Indian Subcontinent
...,...,...,...,...
45767,45768,Strawberries & Cream Frappé,EPICURIOUS,USA
45768,45769,Chocolate Whiskey Bundt Cake,EPICURIOUS,USA
45769,45770,"Wild Rice with Butternut Squash, Leeks, and Corn",EPICURIOUS,USA
45770,45771,Fruit Crumble,EPICURIOUS,USA


In [8]:
df_recipes = df_recipes.rename(columns={'Cuisine': 'Region'})[['Recipe ID', 'Title', 'Region']]

df_recipes

,Recipe ID,Title,Region
0,1,5 spice vegetable fried rice,Indian Subcontinent
1,2,aachar aaloo,Indian Subcontinent
2,3,aadu lassan keri nu athanu,Indian Subcontinent
3,4,aaloo kofta,Indian Subcontinent
4,5,aaloo tamatar subzi,Indian Subcontinent
...,...,...,...
45767,45768,Strawberries & Cream Frappé,USA
45768,45769,Chocolate Whiskey Bundt Cake,USA
45769,45770,"Wild Rice with Butternut Squash, Leeks, and Corn",USA
45770,45771,Fruit Crumble,USA


In [12]:
df_recipes['Region'] = df_recipes['Region'].apply(lambda x: x.replace('Misc.: ', ''))
df_recipes['Region'] = df_recipes['Region'].apply(lambda x: x.replace('DACH Countries', 'Germany, Austria, and Switzerland'))

In [16]:
df_recipes

,Recipe ID,Title,Region
0,1,5 spice vegetable fried rice,Indian Subcontinent
1,2,aachar aaloo,Indian Subcontinent
2,3,aadu lassan keri nu athanu,Indian Subcontinent
3,4,aaloo kofta,Indian Subcontinent
4,5,aaloo tamatar subzi,Indian Subcontinent
...,...,...,...
45767,45768,Strawberries & Cream Frappé,USA
45768,45769,Chocolate Whiskey Bundt Cake,USA
45769,45770,"Wild Rice with Butternut Squash, Leeks, and Corn",USA
45770,45771,Fruit Crumble,USA


In [17]:
df_recipes.to_csv('../data/interim/Recipes.csv', index=False)

## Produção da tabela IngredientsOnRecipes

Abrir dataset "04_Recipe-Ingredients_Aliases" do CulinaryDB que contém os ingredientes que estão em cada receita

In [2]:
import pandas as pd

df_ing = pd.read_csv('04_Recipe-Ingredients_Aliases.csv', sep=',', header=0, encoding='UTF-8')

df_ing

,Recipe ID,Original Ingredient Name,Aliased Ingredient Name,Entity ID
0,1,capsicum,capsicum,362
1,1,green bell pepper,pepper bell,362
2,1,soy sauce,soy sauce,291
3,1,sunflower oil,sunflower,426
4,2,buttermilk,buttermilk,61
...,...,...,...,...
456274,45772,1/2 tsp salt,salt,778
456275,45772,"1 yellow squash, diced",squash yellow,493
456276,45772,1/2 tsp dried thyme,thyme,269
456277,45772,1 tsp Worcestershire sauce,worcestershire sauce,2031


Retirar espaços em branco que estão no ínicio ou no final dos nomes

In [3]:
df_ing['Original Ingredient Name'] = df_ing['Original Ingredient Name'].str.strip()
df_ing['Aliased Ingredient Name'] = df_ing['Aliased Ingredient Name'].str.strip()

Abrir dataset "Food" do FooDB

In [4]:
df_food = pd.read_csv('Food.csv', sep=',', header=0, encoding='UTF-8')

df_food = df_food[['id', 'name']]
df_food

,id,name
0,1,Angelica
1,2,Savoy cabbage
2,3,Silver linden
3,4,Kiwi
4,5,Allium
...,...,...
987,1020,Cape gooseberry
988,1021,Herbal tea
989,1022,Fish oil
990,1023,Taco shell


Para criar a coluna Food ID do IngredientOnFood, foi necessário iterar por cada linha do "04_Recipe-Ingredients_Aliases" e verificar se o alimento dessa linha tinha correspondência com algum alimento do "Food". Se houvesse correspondência, retornava o id do alimento, senão, retornava None

**Obs:** Alguns alimentos não tiveram correspondência, e por isso, ficaram com NaN na coluna no Food ID, precisamos decidir se vamos excluir esses registros ou não

In [7]:
def get_food_id(food_name):
    filtered = df_food[df_food['name'].str.contains(food_name, case=False)]
    if not filtered.empty:
        return filtered.iloc[0]['id']
    return None

df_ing['Food ID'] = df_ing['Aliased Ingredient Name'].apply(get_food_id)

df_ing

,Recipe ID,Original Ingredient Name,Aliased Ingredient Name,Entity ID,Food ID
0,1,capsicum,capsicum,362,233.0
1,1,green bell pepper,pepper bell,362,NaN
2,1,soy sauce,soy sauce,291,716.0
3,1,sunflower oil,sunflower,426,86.0
4,2,buttermilk,buttermilk,61,714.0
...,...,...,...,...,...
456274,45772,1/2 tsp salt,salt,778,666.0
456275,45772,"1 yellow squash, diced",squash yellow,493,NaN
456276,45772,1/2 tsp dried thyme,thyme,269,183.0
456277,45772,1 tsp Worcestershire sauce,worcestershire sauce,2031,NaN


Salvar a nova tabela em csv

In [ ]:
df_ing = df_ing[['Recipe ID', 'Original Ingredient Name', 'Aliased Ingredient Name', 'Food ID']]
df_ing.to_csv('../data/interim/IngredientOnFood.csv', index=False)

# Adicionar a coluna Food ID no IngredientOnFood

In [ ]:
import pandas as pd

df_with_unit = pd.read_csv('../data/interim/Recipe-Ingredients-With-Quantity.csv', sep=',', header=0, encoding='UTF-8')

df_with_unit = df_with_unit.rename(columns={'Recipe ID': 'Recipe_ID', 'Original Ingredient Name': 'Original_Ingredient_Name', 'Aliased Ingredient Name': 'Aliased_Ingredient_Name', 'Entity ID': 'Entity_ID'})

df_with_unit['ID'] = range(1, len(df_with_unit) + 1)

df_with_unit

In [7]:
df_with_food_id = pd.read_csv('../data/interim/IngredientOnFood.csv', sep=',', header=0, encoding='UTF-8')

df_with_food_id['ID'] = range(1, len(df_with_food_id) + 1)

df_with_food_id

,Recipe_ID,Original_Ingredient_Name,Aliased_Ingredient_Name,Food_ID,ID
0,1,capsicum,capsicum,233.0,1
1,1,green bell pepper,pepper bell,NaN,2
2,1,soy sauce,soy sauce,716.0,3
3,1,sunflower oil,sunflower,86.0,4
4,2,buttermilk,buttermilk,714.0,5
...,...,...,...,...,...
456274,45772,1/2 tsp salt,salt,666.0,456275
456275,45772,"1 yellow squash, diced",squash yellow,NaN,456276
456276,45772,1/2 tsp dried thyme,thyme,183.0,456277
456277,45772,1 tsp Worcestershire sauce,worcestershire sauce,NaN,456278


Unir os dois dataframes usando merge

In [ ]:
df_merged = pd.merge(df_with_unit, df_with_food_id[['ID', 'Food_ID']], on='ID', how='left')

df_merged = df_merged.set_index('ID')

df_merged

Retirar as linhas com valores NaN

In [15]:
df_without_nan = df_merged.dropna()[['Recipe_ID', 'Food_ID', 'Aliased_Ingredient_Name', 'Quantity', 'Unit']]

df_without_nan

,Recipe_ID,Food_ID,Aliased_Ingredient_Name,Quantity,Unit
ID,,,,,
17267,2610,8.0,garlic,0.50,teaspoon
17268,2610,206.0,ginger,0.50,teaspoon
17269,2610,561.0,mushroom,4.00,unit
17270,2610,6.0,onion,0.50,teaspoon
17271,2610,6.0,onion,0.25,unit
...,...,...,...,...,...
456272,45772,40.0,pepper,0.12,teaspoon
456274,45772,92.0,potato,0.50,pound
456275,45772,666.0,salt,0.50,teaspoon


Salvar dataset

In [16]:
df_without_nan.to_csv('../data/final/IngredientOnFood.csv', index=False)